# Imports

In [4]:
# pip install xgboost

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import statsmodels.api as smd

import sklearn
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer

%matplotlib inline


# Start

In [5]:
churn_data=pd.read_csv(r'churn_data.csv')
customer_data=pd.read_csv(r'customer_data.csv')
internet_data=pd.read_csv(r'internet_data.csv')

churn=pd.DataFrame(churn_data)
customer=pd.DataFrame(customer_data)
internet=pd.DataFrame(internet_data)

churn.isnull().sum()
internet.isnull().sum()
customer.isnull().sum()

merging_customer_internet=pd.merge(customer,internet)
dataset=pd.merge(churn,merging_customer_internet)

# Pre-Processing

In [6]:
categorical_features=[feature for feature in dataset.columns if dataset[feature].dtype =='object']
numerical_features=['Churn','MonthlyCharges','TotalCharges']

clean_data=dataset.copy()
for feature in ['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']:
    clean_data[feature]=clean_data[feature].replace({'No internet service':'No'})

clean_data=clean_data.iloc[:,1:]
clean_data['MultipleLines']=clean_data['MultipleLines'].replace({'No phone service':'No'})

#check
ind=clean_data[clean_data['TotalCharges']==" "].index
clean_data=clean_data.drop(index=ind)
clean_data['TotalCharges']=clean_data['TotalCharges'].apply(lambda x: float(x) )

# One Hot Encoding

In [7]:
oe=OneHotEncoder()

categorical_features.remove('customerID')
categorical_features.remove('TotalCharges')
categorical_features.remove('Churn')

encodered_data=pd.get_dummies(clean_data[categorical_features])

numerical_features=['tenure','MonthlyCharges','TotalCharges','SeniorCitizen','Churn']

full_data=encodered_data.merge(clean_data[numerical_features],left_index=True, right_index=True)

# check
numerical_features.remove('SeniorCitizen')

# Replacing Yes and No with 0 or 1
full_data['Churn']=np.where(full_data['Churn']=='No',0,1)

# Remove Churn from Full Data set
X=full_data.iloc[:,:-1]

Y=full_data['Churn']

# Scale and Normalize Data

In [8]:
ss=StandardScaler()

col=[colu for colu in clean_data.columns if colu not in numerical_features]

Norm=Normalizer()
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=7)

# XGBOOST

In [9]:
estimate=clean_data['Churn'].value_counts()['No']/clean_data['Churn'].value_counts()['Yes']

In [10]:
from numpy import mean
from xgboost import XGBClassifier
model = XGBClassifier(scale_pos_weight=estimate)
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, X_train, Y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.5f' % mean(scores))

Mean ROC AUC: 0.81581
